
## Interactive Sequential Gaussian Simulation Demonstration


### The Interactive Workflow

Here's a simple workflow for calculating the sequential Gaussian simulation with sequential use of simple kriging estimate and the estimation variance for a local uncertainty model and Monte Carlo simulation.

* we use a 'toy problem' with only 3 data for speed and interpretability of the results

#### Spatial Estimation

Consider the case of making an estimate at some unsampled location, $𝑧(\bf{u}_0)$, where $z$ is the property of interest (e.g. porosity etc.) and $𝐮_0$ is a location vector describing the unsampled location.

How would you do this given data, $𝑧(\bf{𝐮}_1)$, $𝑧(\bf{𝐮}_2)$, and $𝑧(\bf{𝐮}_3)$?

It would be natural to use a set of linear weights to formulate the estimator given the available data.

\begin{equation}
z^{*}(\bf{u}) = \sum^{n}_{\alpha = 1} \lambda_{\alpha} z(\bf{u}_{\alpha})
\end{equation}

We could add an unbiasedness constraint to impose the sum of the weights equal to one.  What we will do is assign the remainder of the weight (one minus the sum of weights) to the global average; therefore, if we have no informative data we will estimate with the global average of the property of interest.

\begin{equation}
z^{*}(\bf{u}) = \sum^{n}_{\alpha = 1} \lambda_{\alpha} z(\bf{u}_{\alpha}) + \left(1-\sum^{n}_{\alpha = 1} \lambda_{\alpha} \right) \overline{z}
\end{equation}

We will make a stationarity assumption, so let's assume that we are working with residuals, $y$. 

\begin{equation}
y^{*}(\bf{u}) = z^{*}(\bf{u}) - \overline{z}(\bf{u})
\end{equation}

If we substitute this form into our estimator the estimator simplifies, since the mean of the residual is zero.

\begin{equation}
y^{*}(\bf{u}) = \sum^{n}_{\alpha = 1} \lambda_{\alpha} y(\bf{u}_{\alpha})
\end{equation}

while satisfying the unbaisedness constraint.  

#### Kriging

Now the next question is what weights should we use?  

We could use equal weighting, $\lambda = \frac{1}{n}$, and the estimator would be the average of the local data applied for the spatial estimate. This would not be very informative.

We could assign weights considering the spatial context of the data and the estimate:

* **spatial continuity** as quantified by the variogram (and covariance function)
* **redundancy** the degree of spatial continuity between all of the available data with themselves 
* **closeness** the degree of spatial continuity between the avaiable data and the estimation location

The kriging approach accomplishes this, calculating the best linear unbiased weights for the local data to estimate at the unknown location.  The derivation of the kriging system and the resulting linear set of equations is available in the lecture notes.  Furthermore kriging provides a measure of the accuracy of the estimate!  This is the kriging estimation variance (sometimes just called the kriging variance).

\begin{equation}
\sigma^{2}_{E}(\bf{u}) = C(0) - \sum^{n}_{\alpha = 1} \lambda_{\alpha} C(\bf{u}_0 - \bf{u}_{\alpha})
\end{equation}

What is 'best' about this estimate? Kriging estimates are best in that they minimize the above estimation variance. 

#### Properties of Kriging

Here are some important properties of kriging:

* **Exact interpolator** - kriging estimates with the data values at the data locations
* **Kriging variance** can be calculated before getting the sample information, as the kriging estimation variance is not dependent on the values of the data nor the kriging estimate, i.e. the kriging estimator is homoscedastic. 
* **Spatial context** - kriging takes into account, furthermore to the statements on spatial continuity, closeness and redundancy we can state that kriging accounts for the configuration of the data and structural continuity of the variable being estimated.
* **Scale** - kriging may be generalized to account for the support volume of the data and estimate. We will cover this later.
* **Multivariate** - kriging may be generalized to account for multiple secondary data in the spatial estimate with the cokriging system. We will cover this later.
* **Smoothing effect** of kriging can be forecast. We will use this to build stochastic simulations later.

#### Spatial Continuity 

**Spatial Continuity** is the correlation between values over distance.

* No spatial continuity – no correlation between values over distance, random values at each location in space regardless of separation distance.

* Homogenous phenomenon have perfect spatial continuity, since all values as the same (or very similar) they are correlated. 

We need a statistic to quantify spatial continuity! A convenient method is the Semivariogram.

#### The Semivariogram

Function of difference over distance.

* The expected (average) squared difference between values separated by a lag distance vector (distance and direction), $h$:

\begin{equation}
\gamma(\bf{h}) = \frac{1}{2 N(\bf{h})} \sum^{N(\bf{h})}_{\alpha=1} (z(\bf{u}_\alpha) - z(\bf{u}_\alpha + \bf{h}))^2  
\end{equation}

where $z(\bf{u}_\alpha)$ and $z(\bf{u}_\alpha + \bf{h})$ are the spatial sample values at tail and head locations of the lag vector respectively.

* Calculated over a suite of lag distances to obtain a continuous function.

* the $\frac{1}{2}$ term converts a variogram into a semivariogram, but in practice the term variogram is used instead of semivariogram.
* We prefer the semivariogram because it relates directly to the covariance function, $C_x(\bf{h})$ and univariate variance, $\sigma^2_x$:

\begin{equation}
C_x(\bf{h}) = \sigma^2_x - \gamma(\bf{h})
\end{equation}

Note the correlogram is related to the covariance function as:

\begin{equation}
\rho_x(\bf{h}) = \frac{C_x(\bf{h})}{\sigma^2_x}
\end{equation}

The correlogram provides of function of the $\bf{h}-\bf{h}$ scatter plot correlation vs. lag offset $\bf{h}$.  

\begin{equation}
-1.0 \le \rho_x(\bf{h}) \le 1.0
\end{equation}

#### Sequential Gaussian Simulation

With sequential Gaussian simulation we build on kriging by:

* adding a random residual with the missing variance

* sequentially adding the simulated values as data to correct the covariance between the simulated values

I have more on this topic at [Simulation YouTube Lecture](https://www.youtube.com/watch?v=3cLqK3lR56Y&list=PLG19vXLQHvSB-D4XKYieEku9GQMQyAzjJ&index=45&t=813s).

#### Load the required libraries

The following code loads the required libraries.

In [1]:
import geostatspy.GSLIB as GSLIB                       # GSLIB utilies, visualization and wrapper
import geostatspy.geostats as geostats                 # GSLIB methods convert to Python    

We will also need some standard packages. These should have been installed with Anaconda 3.

In [2]:
%matplotlib inline
import os                                               # to set current working directory 
import sys                                              # supress output to screen for interactive variogram modeling
import io
import numpy as np                                      # arrays and matrix math
import pandas as pd                                     # DataFrames
import matplotlib.pyplot as plt                         # plotting
from matplotlib.pyplot import cm                        # color maps
from matplotlib.patches import Ellipse                  # plot an ellipse
import math                                             # sqrt operator
import random                                           # random simulation locations
from copy import copy                                   # copy a colormap
from scipy.stats import norm
from ipywidgets import interactive                      # widgets and interactivity
from ipywidgets import widgets                            
from ipywidgets import Layout
from ipywidgets import Label
from ipywidgets import VBox, HBox
from scipy.stats import norm                            # Gaussian distribution
import scipy.stats as stats                             # trimmed statistics

If you get a package import error, you may have to first install some of these packages. This can usually be accomplished by opening up a command window on Windows and then typing 'python -m pip install [package-name]'. More assistance is available with the respective package docs.  

#### Simple, Simple Kriging Function

Let's write a fast Python function to take data points and unknown location and provide the:

* **simple kriging estimate**

* **simple kriging variance / estimation variance**

* **simple kriging weights**

This provides a fast method for small datasets, with less parameters (no search parameters) and the ability to see the simple kriging weights.

* we use it here for fast, flexible application of sequential simulation

* the method will not work with only ones simulation location so we send 2 and only use the first result (the 2nd is always a dummy location in the workflow below.

In [3]:
def simple_simple_krige(df,xcol,ycol,vcol,dfl,xlcol,ylcol,vario,skmean):
# load the variogram
    nst = vario['nst']; pmx = 9999.9
    cc = np.zeros(nst); aa = np.zeros(nst); it = np.zeros(nst)
    ang = np.zeros(nst); anis = np.zeros(nst)
    nug = vario['nug']; sill = nug 
    cc[0] = vario['cc1']; sill = sill + cc[0]
    it[0] = vario['it1']; ang[0] = vario['azi1']; 
    aa[0] = vario['hmaj1']; anis[0] = vario['hmin1']/vario['hmaj1'];
    if nst == 2:
        cc[1] = vario['cc2']; sill = sill + cc[1]
        it[1] = vario['it2']; ang[1] = vario['azi2']; 
        aa[1] = vario['hmaj2']; anis[1] = vario['hmin2']/vario['hmaj2'];    

# set up the required matrices
    rotmat, maxcov = geostats.setup_rotmat(nug,nst,it,cc,ang,pmx)    
    ndata = len(df); a = np.zeros([ndata,ndata]); r = np.zeros(ndata); s = np.zeros(ndata); rr = np.zeros(ndata)
    nest = len(dfl)

    est = np.zeros(nest); var = np.full(nest,sill); weights = np.zeros([nest,ndata])

# Make and solve the kriging matrix, calculate the kriging estimate and variance 
    for iest in range(0,nest):
        for idata in range(0,ndata):
            for jdata in range(0,ndata):
                a[idata,jdata] = geostats.cova2(df[xcol].values[idata],df[ycol].values[idata],df[xcol].values[jdata],df[ycol].values[jdata],
                                        nst,nug,pmx,cc,aa,it,ang,anis,rotmat,maxcov)
            r[idata] = geostats.cova2(df[xcol].values[idata],df[ycol].values[idata],dfl[xlcol].values[iest],dfl[ylcol].values[iest],
                                        nst,nug,pmx,cc,aa,it,ang,anis,rotmat,maxcov)
            rr[idata] = r[idata]
        
        s = geostats.ksol_numpy(ndata,a,r)    
        sumw = 0.0
        for idata in range(0,ndata):                          
            sumw = sumw + s[idata]
            weights[iest,idata] = s[idata]
            est[iest] = est[iest] + s[idata]*df[vcol].values[idata]
            var[iest] = var[iest] - s[idata]*rr[idata]
        est[iest] = est[iest] + (1.0-sumw)*skmean
    return est,var,weights 

#### Interactive Sequential Simulation to Random Points Method

For this first interactive method we will perform sequential simulation:

* at **nsim** random point locations in the area of interest.

The following code includes:

* dashboard with number of simulation locations, variogram model and data locations 

* plots of variogram model, data locations with point scaled by weights and uncertainty distribution at the unknown location

Let's first set up the model area of interest.

In [4]:
csiz = 100; xmn = csiz * 0.5; nx = 10; ymn = csiz * 0.5; ny = 10 
xmin = xmn - csiz * 0.5; xmax = xmin + nx * csiz
ymin = ymn - csiz * 0.5; ymax = ymin + ny * csiz
print('X extents [' + str(xmin) + ',' + str(xmax) + '] and Y entents [' + str(ymin) + ',' + str(ymax) + ']')

X extents [0.0,1000.0] and Y entents [0.0,1000.0]


Now let's set up our dash board.

In [5]:
import warnings; warnings.simplefilter('ignore')

# dashboard: number of simulation locations and variogram parameters
style = {'description_width': 'initial'}
l = widgets.Text(value='                                                   Sequential Simulation, Michael Pyrcz, Professor, The University of Texas at Austin',layout=Layout(width='950px', height='30px'))
nsim = widgets.IntSlider(min = 0, max = 99, value = 5, step = 1, description = 'n',orientation='vertical',
                          layout=Layout(width='25px', height='200px'),continuous_update = False)
nsim.style.handle_color = 'gray'
nug = widgets.FloatSlider(min = 0, max = 1.0, value = 0.0, step = 0.1, description = 'nug',orientation='vertical',
                          layout=Layout(width='25px', height='200px'),continuous_update = False)
nug.style.handle_color = 'gray'
it1 = widgets.Dropdown(options=['Spherical', 'Exponential', 'Gaussian'],value='Spherical',
    description='struct',disabled=False,layout=Layout(width='180px', height='30px'), style=style,continuous_update = False)

azi = widgets.FloatSlider(min=0, max = 360, value = 0, step = 22.5, description = 'azi',
                        orientation='vertical',layout=Layout(width='40px', height='200px'),continuous_update = False)
azi.style.handle_color = 'gray'
hmaj1 = widgets.FloatSlider(min=0.01, max = 10000.0, value = 100.0, step = 25.0, description = 'hmaj',
                        orientation='vertical',layout=Layout(width='40px', height='200px'),continuous_update = False)
hmaj1.style.handle_color = 'gray'
hmin1 = widgets.FloatSlider(min = 0.01, max = 10000.0, value = 100.0, step = 25.0, description = 'hmin',
                        orientation='vertical',layout=Layout(width='40px', height='200px'),continuous_update = False)
hmin1.style.handle_color = 'gray'
uikvar = widgets.HBox([nsim,nug,it1,azi,hmaj1,hmin1],)     

# dashboard: data locations 
x1 = widgets.FloatSlider(min=0.0, max = 1000.0, value = 100.0, step = 1.0, description = 'x1',orientation='horizontal',
                         layout=Layout(width='180px', height='30px'),readout_format = '.0f',style=style,continuous_update = False)
x1.style.handle_color = 'darkblue'
y1 = widgets.FloatSlider(min=0.0, max = 1000.0, value = 100.0, step = 1.0, description = 'y1',orientation='vertical',
                         layout=Layout(width='90px', height='180px'),readout_format = '.0f',style=style,continuous_update = False)
y1.style.handle_color = 'darkblue'
uik1 = widgets.VBox([x1,y1],)

x2 = widgets.FloatSlider(min=0.0, max = 1000.0, value = 500.0, step = 1.0, description = 'x2',orientation='horizontal',
                         layout=Layout(width='180px', height='30px'),readout_format = '.0f',style=style,continuous_update = False)
x2.style.handle_color = 'darkred'
y2 = widgets.FloatSlider(min=0.0, max = 1000.0, value = 800.0, step = 1.0, description = 'y2',orientation='vertical',
                         layout=Layout(width='90px', height='180px'),readout_format = '.0f',style=style,continuous_update = False)
y2.style.handle_color = 'darkred'
uik2 = widgets.VBox([x2,y2],)

x3 = widgets.FloatSlider(min=0.0, max = 1000.0, value = 900.0, step = 1.0, description = 'x3',orientation='horizontal',
                         layout=Layout(width='180px', height='30px'),readout_format = '.0f',style=style,continuous_update = False)
x3.style.handle_color = 'darkorange'
y3 = widgets.FloatSlider(min=0.0, max = 1000.0, value = 200.0, step = 1.0, description = 'y3',orientation='vertical',
                         layout=Layout(width='90px', height='180px'),readout_format = '.0f',style=style,continuous_update = False)
y3.style.handle_color = 'darkorange'
uik3 = widgets.VBox([x3,y3],)

uipars = widgets.HBox([uikvar,uik1,uik2,uik3],) 
uik = widgets.VBox([l,uipars],)

def convert_type(it):
    if it == 'Spherical': 
        return 1
    elif it == 'Exponential':
        return 2
    else: 
        return 3

def f_make_krige(nsim,nug,it1,azi,hmaj1,hmin1,x1,y1,x2,y2,x3,y3): # function to take parameters, make sample and plot
    text_trap = io.StringIO()                           # suppress all text function output to dashboard to avoid clutter 
    sys.stdout = text_trap
    cmap = cm.inferno
    np.random.seed(seed = 73073)                        # ensure same results for all runs
    it1 = convert_type(it1)
    nst = 1; xlag = 10; nlag = int(hmaj1/xlag); c1 = 1.0-nug
    vario = GSLIB.make_variogram(nug,nst,it1,c1,azi,hmaj1,hmin1) # make model object
    index_maj,h_maj,gam_maj,cov_maj,ro_maj = geostats.vmodel(nlag,xlag,azi,vario)   # project the model in the major azimuth                                                  # project the model in the 135 azimuth
    index_min,h_min,gam_min,cov_min,ro_min = geostats.vmodel(nlag,xlag,azi+90.0,vario) # project the model in the minor azimuth
    
    seed = 73073

# make hard data dataframe and hard code the data values
    x = [x1,x2,x3]; y = [y1,y2,y3]; value = [-2.0,0.0,2.0] 
    df = pd.DataFrame({'X':x,'Y':y,'Value':value})
    ndata = len(df); skmean = np.average(df['Value'].values)

# make simulation locations dataframe
    random.seed(a = seed)
    xl = random.sample(range(0, 1000), nsim); 
    random.seed(a = seed+1)
    yl = random.sample(range(0, 1000), nsim); valuel = np.full(nsim,-9999)
    dfl = pd.DataFrame({'X':xl,'Y':yl, 'Value':valuel},dtype=np.single)
    dfl_temp = pd.DataFrame({'X':[-9999,9999],'Y':[-9999,9999], 'Value':[-9999,-9999]},dtype=np.single)
  
    sim = np.zeros(len(dfl)); sk_est = np.zeros(len(dfl)); sk_var = np.zeros(len(dfl)); sk_std = np.zeros(len(dfl))
    sk_weights = np.zeros([ndata,len(dfl)])
     
# perform sequential simulation
    for isim in range(0,len(dfl)):
        dfl_temp.loc[0,'X'] = dfl.loc[isim,'X']; dfl_temp.loc[0,'Y'] = dfl.loc[isim,'Y']; # copy current data to first data / method needs atleast 2 data
        sk_est_temp, sk_var_temp, sk_weights_temp = simple_simple_krige(df,'X','Y','Value',dfl_temp,'X','Y',vario,skmean=skmean)
        sk_est[isim] = sk_est_temp[0]; 
        sk_var[isim] = sk_var_temp[0]; 
        sk_weights[:,isim] = sk_weights_temp[0,:ndata]
        if sk_var[isim] == 0: 
            sk_std[isim] = 0.0
        else:
            sk_std[isim] = math.sqrt(sk_var[isim])
        sim[isim] = norm.rvs(loc=sk_est[isim], scale=sk_std[isim], size=1)[0]  # random seedset at the start 
        #df = df.append({'X': dfl.loc[isim,'X'],'Y': dfl.loc[isim,'Y'],'Value': sim[isim]}, ignore_index=True) # append is removed
        df = pd.concat([df, pd.DataFrame.from_records([{'X': dfl.loc[isim,'X'],'Y': dfl.loc[isim,'Y'],'Value': sim[isim]}])]
            ,ignore_index=True)
        dfl.at[isim,'Value'] = float(sim[isim])
    
# plot the variogram model
    xlag = 10.0; nlag = int(hmaj1/xlag)
    plt.subplot(1,3,1)
    plt.plot([0,hmaj1*1.5],[1.0,1.0],color = 'black')
    plt.plot(h_maj,gam_maj,color = 'black',label = 'Major ' + str(azi))    
    plt.plot(h_min,gam_min,color = 'black',label = 'Minor ' + str(azi+90.0))
    deltas = [22.5, 45, 67.5]; 
    ndelta = len(deltas); hd = np.zeros(ndelta); gamd = np.zeros(ndelta);
    color=iter(cm.plasma(np.linspace(0,1,ndelta)))
    for delta in deltas:
        index,hd,gamd,cov,ro = geostats.vmodel(nlag,xlag,azi+delta,vario);
        c=next(color)
        plt.plot(hd,gamd,color = c,label = 'Azimuth ' + str(azi+delta))
    plt.xlabel(r'Lag Distance $\bf(h)$, (m)')
    plt.ylabel(r'$\gamma \bf(h)$')
    plt.title('Interpolated NSCORE Porosity Variogram Models')
    plt.xlim([0,hmaj1*1.5])
    plt.ylim([0,1.4])
    plt.legend(loc='upper left')

# plot the data and simulated values on a scatter plot 
    sk_weights_avg = np.mean(sk_weights,axis = 1)
    plt.subplot(1,3,2)
    for idata in range(0,len(df)):
        if idata < ndata:
            plt.scatter([df.loc[idata,'X']],[df.loc[idata,'Y']],marker='^',
                        c = [df.loc[idata,'Value']], cmap = cmap, vmin = -2.0, vmax = 2.0, edgecolors = 'black',
                        s = 100,label = 'Original Data')
        else: 
            plt.scatter([df.loc[idata,'X']],[df.loc[idata,'Y']],
                        c = [df.loc[idata,'Value']], cmap = cmap, vmin = -2.0, vmax = 2.0, edgecolors = 'black',
                        label = 'Simulated Values')
    
    ax = plt.gca()
    plt.xlabel('X(m)'); plt.ylabel('Y(m)')
    plt.title('Sequential Simulation - Data and Unknown Locations')
    plt.xlim([0,1000])
    plt.ylim([0,1000])
    plt.colorbar()

    if nsim < 10:
        for i, txt in enumerate(np.round(dfl['Value'].values,2)):
            plt.annotate(txt, (dfl.loc[i,'X']-40, dfl.loc[i,'Y']-40))        

    ellipse = Ellipse((500, 500),width=hmin1*2.0,height=hmaj1*2.0,angle = 360-azi,facecolor='none',edgecolor='black',alpha = 1.0,linestyle='--')
    ax = plt.gca()
    ax.add_patch(ellipse)

# plot the distribution of the simulated values   
    plt.subplot(1,3,3)
    plt.hist(sim,bins = np.linspace(-3.0,3.0,20),alpha=0.2,color="red",edgecolor="black")
    plt.xlim([-3.0,3.0]); plt.ylim([0,nsim/2])
    plt.title('Uncertainty Model at Unknown Location')
    plt.xlabel('Value'); plt.ylabel('Frequency')
    
    ax = plt.gca()
    ax.annotate('Simulations: Mean = ' + str(np.round(np.average(sim),2)), (-2.8, nsim*0.05))
    ax.annotate('Simulations: Standard Deviation = ' + str(np.round(np.std(sim),2)), (-2.8, nsim *0.02))
    plt.subplots_adjust(left=0.0, bottom=0.0, right=2.2, top=0.9, wspace=0.3, hspace=0.3)
    plt.show()
    
# connect the function to make the samples and plot to the widgets    
interactive_plot = widgets.interactive_output(f_make_krige, {'nsim':nsim,'nug':nug, 'it1':it1, 'azi':azi, 'hmaj1':hmaj1, 'hmin1':hmin1, 
                                                      'x1':x1, 'y1':y1, 'x2':x2, 'y2':y2, 'x3':x3, 'y3':y3,})
interactive_plot.clear_output(wait = True)               # reduce flickering by delaying plot updating

### Interactive Sequential Simulation to Random Points Demostration

* select the variogram model and the data locations and observe the outputs from sequential simulation 

### The Inputs

Select the variogram model and the data locations:

* **nug**: nugget effect

* **c1 **: contributions of the sill

* **hmaj1 / hmin1 **: range in the major and minor direction

* **(x1, y1),...(x3,y3) **: spatial data locations  

In [6]:
display(uik, interactive_plot)                            # display the interactive plot

Output(outputs=({'output_type': 'display_data', 'metadata': {}, 'data': {'text/plain': '<Figure size 640x480 w…

#### Interactive Sequential Simulation to a Regular Grid Method

Let's repeat the previous interactive demonstration, but this time we will simulate on a random set of nodes on a regular grid with our point data.

* this is more similar to current practice with most spatial modeling software

The following code includes:

* dashboard with number of simulated nodes, variogram model and data locations 

* plot of the point data and the simulated model on a regular grid

In [7]:
import warnings; warnings.simplefilter('ignore')

# dashboard: number of simulation grid nodes and the variogram model
style = {'description_width': 'initial'}
l = widgets.Text(value='                                              Sequential Simulation, Michael Pyrcz, Associate Professor, The University of Texas at Austin',layout=Layout(width='950px', height='30px'))
nsim = widgets.IntSlider(min = 0, max = 100, value = 5, step = 1, description = 'n',orientation='vertical',
                          layout=Layout(width='40px', height='200px'),continuous_update=False)
nsim.style.handle_color = 'gray'
nug = widgets.FloatSlider(min = 0, max = 1.0, value = 0.0, step = 0.1, description = 'nug',orientation='vertical',
                          layout=Layout(width='25px', height='200px'),continuous_update=False)
nug.style.handle_color = 'gray'
it1 = widgets.Dropdown(options=['Spherical', 'Exponential', 'Gaussian'],value='Spherical',
    description='struct',disabled=False,layout=Layout(width='180px', height='30px'), style=style)

seed = widgets.IntText(value=73074,description='Seed:',disabled=False,layout=Layout(width='180px', height='30px'),continuous_update=False)

azi = widgets.FloatSlider(min=0, max = 360, value = 0, step = 22.5, description = 'azi',
                        orientation='vertical',layout=Layout(width='40px', height='200px'),continuous_update=False)
azi.style.handle_color = 'gray'
hmaj1 = widgets.FloatSlider(min=0.01, max = 10000.0, value = 100.0, step = 25.0, description = 'hmaj',
                        orientation='vertical',layout=Layout(width='40px', height='200px'),continuous_update=False)
hmaj1.style.handle_color = 'gray'
hmin1 = widgets.FloatSlider(min = 0.01, max = 10000.0, value = 100.0, step = 25.0, description = 'hmin',
                        orientation='vertical',layout=Layout(width='40px', height='200px'),continuous_update=False)
hmin1.style.handle_color = 'gray'

uikvarb = widgets.VBox([it1,seed],)

uikvar = widgets.HBox([nsim,nug,uikvarb,azi,hmaj1,hmin1],)                   # basic widget formatting   

# dashboard: data locations
x1 = widgets.FloatSlider(min=0.0, max = 1000.0, value = 100.0, step = 1.0, description = 'x1',orientation='horizontal',
                         layout=Layout(width='180px', height='30px'),readout_format = '.0f',style=style,continuous_update=False)
x1.style.handle_color = 'blue'
y1 = widgets.FloatSlider(min=0.0, max = 1000.0, value = 100.0, step = 1.0, description = 'y1',orientation='vertical',
                         layout=Layout(width='90px', height='180px'),readout_format = '.0f',style=style,continuous_update=False)
y1.style.handle_color = 'blue'
uik1 = widgets.VBox([x1,y1],)

x2 = widgets.FloatSlider(min=0.0, max = 1000.0, value = 500.0, step = 1.0, description = 'x2',orientation='horizontal',
                         layout=Layout(width='180px', height='30px'),readout_format = '.0f',style=style,continuous_update=False)
x2.style.handle_color = 'red'
y2 = widgets.FloatSlider(min=0.0, max = 1000.0, value = 800.0, step = 1.0, description = 'y2',orientation='vertical',
                         layout=Layout(width='90px', height='180px'),readout_format = '.0f',style=style,continuous_update=False)
y2.style.handle_color = 'red'
uik2 = widgets.VBox([x2,y2],)

x3 = widgets.FloatSlider(min=0.0, max = 1000.0, value = 900.0, step = 1.0, description = 'x3',orientation='horizontal',
                         layout=Layout(width='180px', height='30px'),readout_format = '.0f',style=style,continuous_update=False)
x3.style.handle_color = 'darkorange'
y3 = widgets.FloatSlider(min=0.0, max = 1000.0, value = 200.0, step = 1.0, description = 'y3',orientation='vertical',
                         layout=Layout(width='90px', height='180px'),readout_format = '.0f',style=style,continuous_update=False)
y3.style.handle_color = 'darkorange'
uik3 = widgets.VBox([x3,y3],)

uipars = widgets.HBox([uikvar,uik1,uik2,uik3],) 
uik = widgets.VBox([l,uipars],)

def convert_type(it):
    if it == 'Spherical': 
        return 1
    elif it == 'Exponential':
        return 2
    else: 
        return 3

def f_make_krige2(nsim,nug,it1,seed,azi,hmaj1,hmin1,x1,y1,x2,y2,x3,y3): # function to take parameters, make sample and plot
    text_trap = io.StringIO()                           # suppress all text function output to dashboard to avoid clutter 
    sys.stdout = text_trap
    cmap = cm.inferno                                                   
    it1 = convert_type(it1)
    nst = 1; xlag = 10; nlag = int(hmaj1/xlag); c1 = 1.0-nug
    vario = GSLIB.make_variogram(nug,nst,it1,c1,azi,hmaj1,hmin1) # make model object
    index_maj,h_maj,gam_maj,cov_maj,ro_maj = geostats.vmodel(nlag,xlag,azi,vario)   # project the model in the major azimuth                                                  # project the model in the 135 azimuth
    index_min,h_min,gam_min,cov_min,ro_min = geostats.vmodel(nlag,xlag,azi+90.0,vario) # project the model in the minor azimuth
    
# make data dataframe
    x = [x1,x2,x3]; y = [y1,y2,y3]; value = [-1.5,0.0,1.5] 
    df = pd.DataFrame({'X':x,'Y':y,'Value':value})
    ndata = len(df); skmean = np.average(df['Value'].values)

# make simulation nodes dataframe   
    random.seed(a = seed)                               # ensure same results for all runs, you can sequentially add / remove nodes
    if nsim == 100:
        icelll = np.linspace(0, nx*ny-1, 100) 
        random.shuffle(icelll)
    else:
        random.seed(seed)
        icelll = np.asarray(random.sample(range(0, nx*ny-1), nsim),dtype = np.int32)
    iyl = np.around(icelll / nx-0.49,0); yl = iyl * csiz + ymn
    ixl = np.around(icelll - iyl * nx , 0); xl = ixl * csiz + xmn
    valuel = np.full(nsim,-9999)
    dfl = pd.DataFrame({'X':xl,'Y':yl, 'Value':valuel},dtype=np.single)
    dfl_temp = pd.DataFrame({'X':[-9999,9999],'Y':[-9999,9999], 'Value':[-9999,-9999]},dtype=np.single)
  
    np.random.seed(seed = seed)
    sim = np.zeros(len(dfl)); sk_est = np.zeros(len(dfl)); sk_var = np.zeros(len(dfl)); sk_std = np.zeros(len(dfl))
    sk_weights = np.zeros([ndata,len(dfl)])

# perform sequential simulation
    for isim in range(0,len(dfl)):
        dfl_temp.loc[0,'X'] = dfl.loc[isim,'X']; dfl_temp.loc[0,'Y'] = dfl.loc[isim,'Y']; # copy current data to first data / method needs atleast 2 data
        sk_est_temp, sk_var_temp, sk_weights_temp = simple_simple_krige(df,'X','Y','Value',dfl_temp,'X','Y',vario,skmean=skmean)
        sk_est[isim] = sk_est_temp[0]; 
        sk_var[isim] = sk_var_temp[0]; 
        sk_weights[:,isim] = sk_weights_temp[0,:ndata]
        if sk_var[isim] == 0: 
            sk_std[isim] <= 0.0
        else:
            sk_std[isim] = math.sqrt(sk_var[isim])
        sim[isim] = norm.rvs(loc=sk_est[isim], scale=sk_std[isim], size=1)[0]  # random seedset at the start 
  
        #df = df.append({'X': dfl.loc[isim,'X'],'Y': dfl.loc[isim,'Y'],'Value': sim[isim]}, ignore_index=True) # append has been removed
        df = pd.concat([df, pd.DataFrame.from_records([{'X': dfl.loc[isim,'X'],'Y': dfl.loc[isim,'Y'],'Value': sim[isim]}])]
                      ,ignore_index=True)
    
        dfl.at[isim,'Value'] = float(sim[isim])

# make the 2D simulated model on a regular grid        
    plt.subplot(121)
    model = np.full([ny,nx],-999.9)            
    for idata in range(len(df)-1,-1,-1):
        ix = int(df.loc[idata,'X']/csiz); iy = int(df.loc[idata,'Y']/csiz);
        model[ny - iy - 1, ix] = df.loc[idata,'Value']
        
    ax = plt.gca()
    plt.xlabel('X(m)'); plt.ylabel('Y(m)')
    plt.title('Sequential Simulation - Data, Simulated Values and Random Path')  
    palette = copy(plt.cm.inferno)
    palette.set_under('r', 0.0)
    palette.set_over('r', 0.0)
    im = plt.imshow(model,interpolation = None,extent = [0,1000,0,1000], vmin = -3.0, vmax = 3.0,cmap = palette)
    plt.scatter(df['X'].values[:ndata],df['Y'].values[:ndata],marker='^',c=df['Value'].values[:ndata], vmin = -2.0, vmax = 2.0, cmap = cmap, edgecolors = 'black',s = 300,label = 'Original Data')
    plt.xlim([0,1000]); plt.ylim([0,1000])
    for idata in range(len(df)-1,-1,-1):
        x = df.loc[idata,'X'];y = df.loc[idata,'Y']
        ix = int(x/csiz); iy = int(y/csiz)
        xc = csiz*ix + csiz*0.45; yc = csiz*iy + csiz*0.5; 
#         if idata < 3:
#             #plt.annotate('D'+str(idata+1),[xc-15,yc],color='white',weight='bold')
#         else:
        if idata > 2:
            plt.annotate(idata-2,[xc-10,yc],color='white')

    cbar = plt.colorbar(im,ax = plt.gca()) # Similar to fig.colorbar(im, cax = cax)
    plt.gca().set_aspect('auto')
    cbar.set_label('Simulated Values')
    
# plot the variogram modle for visualization    
    ellipse1 = Ellipse((x1, y1),width=hmin1*2.0,height=hmaj1*2.0,angle = 360-azi,facecolor='none',edgecolor='darkblue',lw=3,alpha = 0.6)
    ellipse2 = Ellipse((x2, y2),width=hmin1*2.0,height=hmaj1*2.0,angle = 360-azi,facecolor='none',edgecolor='darkred',lw=3,alpha = 0.6)
    ellipse3 = Ellipse((x3, y3),width=hmin1*2.0,height=hmaj1*2.0,angle = 360-azi,facecolor='none',edgecolor='darkorange',lw=3,alpha = 0.6)
    ax = plt.gca()
    ax.add_patch(ellipse1); ax.add_patch(ellipse2); ax.add_patch(ellipse3);
    
    x_values = np.linspace(-3.0,3.0,100)                                          # get an array of x values
    p_values = norm.pdf(x_values, loc = 0.0, scale = 1.0)    
    
    plt.subplot(122)
    plt.hist(model.flatten(),color='red',alpha=0.8,edgecolor='black',bins=np.linspace(-3,3,10),density =True)
    plt.plot(x_values,p_values,color='red')
    plt.xlim(-3,3); plt.ylim(0,0.6)
    plt.title('Distribution of Seuqential Gaussian Simulated Values')
    plt.xlabel('Simulated Gaussian Values'); plt.ylabel('Normalized Frequence')
    
    plt.gca().annotate('Simulation Mean = ' + str(np.round(stats.tmean(model.flatten(),limits=(-5,5)),2)), (0.9, 0.55))
    plt.gca().annotate('Simulation StDev. = ' + str(np.round(stats.tstd(model.flatten(),limits=(-3,3)),2)), (0.9, 0.52))
    
    plt.subplots_adjust(left=0.0, bottom=0.0, right=2.0, top=1.2, wspace=0.3, hspace=0.3)
    plt.show()
    
# connect the function to make the samples and plot to the widgets    
interactive_plot = widgets.interactive_output(f_make_krige2, {'nsim':nsim,'nug':nug, 'it1':it1,'seed':seed,'azi':azi, 'hmaj1':hmaj1, 'hmin1':hmin1, 
                                                      'x1':x1, 'y1':y1, 'x2':x2, 'y2':y2, 'x3':x3, 'y3':y3,})
interactive_plot.clear_output(wait = True)               # reduce flickering by delaying plot updating

### Interactive Sequential Simulation to Model Regular Grid Demonstration

* select the variogram model and the data locations and observe the outputs from sequential simulation 

### The Inputs

Select the simulation nodes, variogram model and the data locations:

* **nsim**: number of simulated nodes, for computational speed up use less nodes

* **nug**: nugget effect

* **c1 **: contributions of the sill

* **hmaj1 / hmin1 **: range in the major and minor direction

* **(x1, y1),...(x3,y3) **: spatial data locations  

In [8]:
display(uik, interactive_plot)                            # display the interactive plot

Output(outputs=({'output_type': 'display_data', 'metadata': {}, 'data': {'text/plain': '<Figure size 640x480 w…

#### Comments

This was an interactive demonstration of sequential Gaussian simulation for spatial data analytics. Much more could be done, I have other demonstrations on the basics of working with DataFrames, ndarrays, univariate statistics, plotting data, declustering, data transformations and many other workflows available at https://github.com/GeostatsGuy/PythonNumericalDemos and https://github.com/GeostatsGuy/GeostatsPy. 